# Exercises of Week 4

*These exercises will not be graded. Solutions will be made available, but it is strongly advised that you try on your own first.*

*Make sure that any function you write has a docstring, and comments where appropriate.*

## Question 1

In this exercise you will download returns on 49 industry portfolios and create sorted portfolios from them to analyze if there is an anomaly or mispricing. The exercises will ask you to form $P=7$ portfolios sorted on momentum, and compute abnormal returns based on the Fama-Frech three factor model. 

Feel free to explore other options and see how they impact the results! In particular, you can consider the following choices:
* *Characteristic*: size, value, momentum, reversal, or ...
* *Number of sorts*: recommended to start with univariate sort.
* *Number of portfolios $P$*.
* *Value weighted or equal weighted portfolio returns*.
* *Asset pricing model*: consider for example CAPM or Fama-French three factor model (FF3)

**1.1.** Import the relevant packages:

In [16]:
import pandas as pd
import pandas_datareader.data as web
import pandas_datareader 
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline

**1.2.** Download monthly value weighted returns on the 49 industry portfolios from [Kenneth French's website](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html) for the period January 2010 to September 2021: 
download the data set `'49_industry_portfolios'` from source `'famafrench'` via `pandas-datareader`. The output of DataReader will be a `dict` object ([documentation](https://docs.python.org/3/library/stdtypes.html?highlight=dict#dict)). Use `dict['DESCR']` to see the data description. Index into the `dict` object to select the monthly value weighted returns and store it in a new DataFrame: use `dict[key]`, where `key` will be the index number as given in the data description.

*Note:* The column headers of the returns DataFrame are all strings of length 5. An additional space is added if the industry name has less than 5 letters, e.g. 'Food ' instead of 'Food'. The spaces can be removed using the following code: `R.columns = [x.strip(' ') for x in R.columns]`, assuming that your DataFrame is called `R`. The function `strip(' ')` removes any leading and trailing spaces. It is not necessary to do this for the exercise, but avoids unexpected behavior when trying to select a column.

In [26]:
# Check available datasets
# pandas_datareader.famafrench.get_available_datasets()

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [35]:
startdate = pd.datetime(2000, 1, 1)
enddate = pd.datetime(2021, 10, 30)  # Up to end of last month

df = pd.DataFrame() # Empty DataFrame
print("Finished downloading data for:")

dict1 = web.DataReader('49_industry_portfolios', 'famafrench', start=startdate, end=enddate)
dict1
# 2nd way
# pandas_datareader.famafrench.FamaFrenchReader(symbols, start=None, end=None, retry_count=3, pause=0.1, timeout=30, session=None, freq=None)

<ipython-input-35-733d295b2eb4>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  startdate = pd.datetime(2000, 1, 1)
<ipython-input-35-733d295b2eb4>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  enddate = pd.datetime(2021, 10, 30)  # Up to end of last month


Finished downloading data for:


In [37]:
dict1.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 'DESCR'])

In [41]:
dict1[0]

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2000-01,-4.50,-10.33,19.70,-2.28,-8.62,-13.81,3.76,-0.62,-6.51,-11.50,...,-13.15,-9.56,7.59,-14.51,-7.39,-2.55,-7.27,-0.69,-6.80,0.04
2000-02,8.16,-7.08,-8.28,-11.64,-4.01,0.42,-2.31,-0.61,-11.57,-12.73,...,-12.60,-5.35,1.76,-3.93,-12.03,-11.30,-12.76,2.27,6.55,-2.15
2000-03,4.26,10.76,-0.29,0.13,5.11,7.69,10.70,13.12,-14.26,25.38,...,12.43,12.87,7.03,14.38,15.98,15.04,22.87,3.28,14.71,-2.00
2000-04,-7.61,-4.08,-0.80,3.68,3.79,0.27,2.35,-7.54,4.29,4.30,...,-8.21,3.48,-2.60,-5.60,3.82,-2.85,-0.40,-3.71,-11.14,9.24
2000-05,-2.47,18.06,-7.24,11.87,19.51,1.15,1.26,-6.34,2.79,-5.04,...,-2.34,-3.63,1.30,-3.04,-4.21,7.95,7.31,-2.71,-6.81,10.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05,-6.12,3.94,1.99,1.99,2.04,2.94,-0.86,2.90,0.29,-0.57,...,-3.53,2.14,1.07,-2.82,-1.25,2.73,2.91,1.28,4.55,4.88
2021-06,-2.35,-3.06,-1.45,-0.98,1.99,12.01,1.56,-0.23,-0.26,8.82,...,-3.59,-4.29,-1.82,3.73,-0.77,-2.90,-3.38,-0.99,1.63,-3.66
2021-07,-3.06,-2.29,4.53,1.09,0.96,-3.07,-7.45,-2.55,3.91,4.54,...,-1.42,-4.90,2.38,-0.79,4.50,-0.25,0.11,4.26,1.71,-0.12


In [42]:
dict1[1]

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2000-01,4.66,0.62,4.11,3.52,12.73,9.24,3.21,2.07,2.93,-1.02,...,-7.57,-1.11,6.91,-4.91,2.06,-3.35,-3.05,4.00,-1.91,9.49
2000-02,51.97,2.16,-2.73,-1.62,-6.25,6.82,4.11,4.55,5.94,1.29,...,-1.81,0.60,4.66,0.92,1.85,-3.75,-1.97,-0.44,1.98,3.31
2000-03,-3.73,5.09,6.52,0.92,-3.01,2.59,0.75,10.62,1.33,8.13,...,9.41,6.08,-0.18,6.56,1.65,1.45,5.87,1.89,0.30,5.35
2000-04,-4.26,-4.03,-1.24,0.27,7.26,-8.75,-8.45,-10.05,-4.29,-3.36,...,-6.09,-2.39,-6.23,-7.19,0.36,-0.30,-3.90,-5.63,-2.05,-12.49
2000-05,-4.88,5.03,0.47,2.33,12.00,-7.27,-6.35,-4.32,-4.48,-7.48,...,-5.32,-4.88,-6.31,-6.86,-2.69,0.98,3.81,-4.74,-2.11,-3.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05,-2.38,4.38,8.38,8.43,-2.60,4.01,6.51,3.72,1.82,-1.08,...,0.00,1.12,10.30,5.62,-1.53,3.80,0.58,5.94,4.92,-0.71
2021-06,0.61,-0.53,4.18,-2.24,3.26,3.72,1.57,3.10,-0.08,1.93,...,-3.43,-5.68,-1.01,3.64,1.55,-2.89,-1.60,1.72,0.66,2.06
2021-07,1.75,-3.32,-1.82,-2.59,5.86,-4.45,-9.24,-1.68,-7.26,-3.85,...,-1.89,-3.28,-1.78,-3.95,-2.97,-0.84,-0.93,-0.63,-0.44,-4.48


In [43]:
dict1[2]

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2000,-5.99,27.10,26.31,17.90,102.39,-12.72,-6.07,0.05,-16.67,20.63,...,-32.35,6.43,38.11,-18.53,0.40,22.26,41.08,4.41,15.44,12.01
2001,21.12,4.48,5.51,-16.05,11.11,26.82,24.18,3.56,2.04,-0.33,...,24.97,-3.18,-26.82,12.39,1.05,-0.54,-8.74,26.05,-14.58,-16.37
2002,-5.18,-0.40,12.57,-1.56,-7.21,-4.36,-5.15,4.15,4.06,-2.79,...,13.30,-0.56,-19.35,-22.01,-17.07,-8.60,-14.55,-2.64,-25.80,-36.02
2003,41.59,7.56,1.20,16.97,41.58,35.10,36.07,19.50,15.48,39.03,...,14.78,22.97,29.39,29.15,37.84,34.07,23.93,39.70,48.23,32.28
2004,15.71,20.58,6.54,-10.00,20.68,14.34,20.02,2.96,14.17,24.83,...,38.47,19.00,14.10,11.09,25.97,13.50,17.01,49.58,11.22,19.47
2005,-2.97,-6.14,6.09,2.89,25.62,-15.24,-4.47,-13.41,-0.82,3.96,...,1.28,6.50,9.02,5.25,8.28,0.64,19.98,6.92,20.49,-0.47
2006,23.80,19.99,12.98,15.04,22.93,20.43,32.69,20.03,16.27,22.74,...,19.60,9.90,13.87,8.15,23.79,17.09,7.67,1.78,34.41,13.79
2007,100.16,3.28,29.73,23.80,17.95,-21.49,3.42,-21.02,10.60,-9.52,...,29.69,0.05,1.78,-6.45,1.12,-24.33,-2.30,-19.14,-4.19,4.44
2008,-36.78,-17.30,-43.69,-16.54,-24.30,-31.37,-69.14,-60.36,-20.91,-33.01,...,-29.24,-23.85,-35.39,-20.90,-18.25,-45.43,-49.50,-63.83,-59.62,-50.15


In [45]:
dict1[5]

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2000-01,226.34,2078.29,1512.17,10342.44,12584.58,295.90,1688.24,3670.86,3176.43,509.18,...,799.34,960.60,596.22,3034.12,800.97,2243.73,2628.20,165.93,878.56,687.13
2000-02,216.11,1880.01,1810.00,10106.77,11499.46,254.82,1768.58,3647.20,2996.15,455.96,...,727.59,878.49,643.85,2615.36,741.71,2197.89,2447.81,164.77,822.83,698.60
2000-03,233.68,1759.91,1659.83,8923.68,11038.70,255.54,1799.65,3685.18,2645.56,406.80,...,634.35,831.94,656.45,2524.75,652.38,1969.78,2175.67,170.58,891.90,677.76
2000-04,243.28,1944.70,1652.41,8907.69,11335.75,274.63,2012.71,4246.47,2267.60,509.14,...,712.67,952.69,704.23,2895.43,767.18,2285.10,2609.90,175.92,1031.41,664.20
2000-05,224.77,1878.94,1639.17,9235.26,14605.66,278.84,2083.14,3925.40,2428.17,537.18,...,654.18,998.66,690.55,2740.95,796.35,2219.37,2611.06,173.22,918.44,725.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05,5897.59,12499.85,36828.51,38721.82,79163.17,5519.68,10772.14,2650.84,18334.74,13411.12,...,10636.08,15907.87,5080.10,31405.47,11807.06,8305.44,15662.63,2580.27,12430.76,27134.20
2021-06,5522.99,12959.90,37562.43,39473.46,80775.54,5679.52,10679.30,2725.67,18376.41,13316.53,...,10238.73,16212.06,5189.59,30496.70,11625.02,8546.64,16270.22,2613.36,12977.04,28925.52
2021-07,5338.06,11732.17,36893.05,34698.45,81278.84,6185.43,11107.97,2598.73,16353.62,12709.91,...,9130.28,15525.16,4766.97,29231.74,13043.93,8147.27,14648.82,2827.96,13661.81,5154.93


In [33]:
dict1['DESCR']

'49 industry portfolios\n----------------------\n\nThis file was created by CMPT_IND_RETS using the 202109 CRSP database. It contains value- and equal-weighted returns for 49 industry portfolios. The portfolios are constructed at the end of June. The annual returns are from January to December. Missing data are indicated by -99.99 or -999. Copyright 2021 Kenneth R. French\n\n  0 : Average Value Weighted Returns -- Monthly (261 rows x 49 cols)\n  1 : Average Equal Weighted Returns -- Monthly (261 rows x 49 cols)\n  2 : Average Value Weighted Returns -- Annual (21 rows x 49 cols)\n  3 : Average Equal Weighted Returns -- Annual (21 rows x 49 cols)\n  4 : Number of Firms in Portfolios (261 rows x 49 cols)\n  5 : Average Firm Size (261 rows x 49 cols)\n  6 : Sum of BE / Sum of ME (22 rows x 49 cols)\n  7 : Value-Weighted Average of BE/ME (22 rows x 49 cols)'

**1.3.** We will need size information to compute value weighted portfolio returns. For that, we will calculate size per industry as the product of the average firm size in the industry times the number of firms in the industry. Both, average size and number of firms dataframes, are available in the same `dict` object as the returns. Also store size in a new DataFrame. 

In [50]:
size_per_industry = dict1[5]*dict1[4];size_per_industry

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2000-01,3847.78,193280.97,13609.53,237876.12,62922.90,15978.60,162071.04,198226.44,292231.56,43280.30,...,15187.46,126799.20,141900.36,952713.68,99320.28,1238538.96,509870.80,7632.78,365480.96,41914.93
2000-02,3673.87,172960.92,16290.00,232455.71,57497.30,13760.28,168015.10,196948.80,272649.65,38300.64,...,13096.62,113325.21,152592.45,813376.96,91972.04,1204443.72,472427.33,7579.42,339828.79,41916.00
2000-03,3972.56,160151.81,14938.47,205244.64,55193.50,13799.16,163768.15,195314.54,240745.96,33357.60,...,11418.30,106488.32,154922.20,780147.75,80895.12,1065650.98,409025.96,7505.52,360327.60,38632.32
2000-04,4135.76,176967.70,14871.69,204876.87,56678.75,14830.02,181143.90,220816.44,206351.60,41749.48,...,12828.06,120038.94,161972.90,891792.44,93595.96,1222528.50,485441.40,7740.48,412564.00,37859.40
2000-05,3821.09,169104.60,14752.53,212410.98,58422.64,14778.52,185399.46,204120.80,213678.96,43511.58,...,11775.24,123833.84,157445.40,841471.65,97154.70,1185143.58,483046.10,7448.46,365539.12,41348.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05,41283.13,537493.55,294628.08,309774.56,237489.51,132472.32,420113.46,34460.92,715054.86,402333.60,...,95724.72,986287.94,431808.50,3862872.81,625774.18,3438452.16,1456624.59,82568.64,1081476.12,1628052.00
2021-06,38660.93,557275.70,300499.44,315787.68,242326.62,136308.48,416492.70,35433.71,716679.99,399495.90,...,92148.57,1005147.72,435925.56,3751094.10,616126.06,3521215.68,1496860.24,83627.52,1129002.48,1706605.68
2021-07,37366.42,539679.82,295144.40,312286.05,243836.52,167006.61,477642.71,36382.22,670498.42,394007.21,...,91302.80,993610.24,433794.27,4004748.38,704372.22,3454442.48,1464882.00,93322.68,1229562.90,1685662.11


**1.4.** Before sorting and grouping the assets into portfolios, we need to compute or read the characteristic(s). Momentum is defined as the return from month $t-12$ to $t-2$.

In [58]:
r

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2000-01,-4.604394,-10.903392,17.981843,-2.306394,-9.014355,-14.861602,3.691035,-0.621930,-6.731571,-12.216763,...,-14.098769,-10.048354,7.315752,-15.677078,-7.677306,-2.583076,-7.547814,-0.692392,-7.042246,0.039992
2000-02,7.844143,-7.343128,-8.642973,-12.375081,-4.092617,0.419120,-2.337099,-0.611868,-12.295891,-13.616342,...,-13.467490,-5.498431,1.744691,-4.009309,-12.817434,-11.991030,-13.650724,2.244619,6.344417,-2.173449
2000-03,4.171759,10.219551,-0.290421,0.129916,4.983723,7.408654,10.165365,12.327902,-15.385072,22.617894,...,11.716062,12.106653,6.793898,13.435605,14.824758,14.010971,20.595670,3.227356,13.723702,-2.020271
2000-04,-7.915144,-4.165568,-0.803217,3.613905,3.719944,0.269636,2.322813,-7.839407,4.200529,4.210118,...,-8.566683,3.420817,-2.634398,-5.762911,3.748844,-2.891401,-0.400802,-3.780571,-11.810809,8.837711
2000-05,-2.501016,16.602278,-7.515467,11.216730,17.822986,1.143438,1.252128,-6.549898,2.751789,-5.171444,...,-2.367813,-3.697524,1.291623,-3.087166,-4.301189,7.649797,7.055166,-2.747398,-7.052977,9.939274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05,-6.315282,3.864362,1.970458,1.970458,2.019471,2.897611,-0.863719,2.858746,0.289580,-0.571631,...,-3.593811,2.117424,1.064316,-2.860526,-1.257878,2.693400,2.868463,1.271877,4.449524,4.764665
2021-06,-2.378053,-3.107796,-1.460615,-0.984834,1.970458,11.341797,1.547957,-0.230265,-0.260339,8.452496,...,-3.656026,-4.384740,-1.836766,3.662118,-0.772980,-2.942881,-3.438443,-0.994933,1.616858,-3.728658
2021-07,-3.107796,-2.316628,4.430393,1.084102,0.955421,-3.118112,-7.742115,-2.583076,3.835495,4.439959,...,-1.430178,-5.024122,2.352120,-0.793137,4.401689,-0.250313,0.109940,4.171759,1.695544,-0.120072


In [65]:
r.rolling(2).sum()

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02,3.239749,-18.246520,9.338870,-14.681475,-13.106972,-14.442482,1.353937,-1.233798,-19.027461,-25.833106,...,-27.566260,-15.546785,9.060443,-19.686387,-20.494740,-14.574106,-21.198539,1.552227,-0.697829,-2.133457
2000-03,12.015902,2.876423,-8.933394,-12.245165,0.891107,7.827775,7.828267,11.716034,-27.680963,9.001552,...,-1.751428,6.608222,8.538590,9.426296,2.007324,2.019941,6.944946,5.471975,20.068119,-4.193720
2000-04,-3.743384,6.053984,-1.093638,3.743820,8.703668,7.678290,12.488178,4.488495,-11.184543,26.828012,...,3.149379,15.527470,4.159501,7.672694,18.573602,11.119570,20.194868,-0.553215,1.912893,6.817440
2000-05,-10.416160,12.436711,-8.318685,14.830634,21.542930,1.413074,3.574941,-14.389305,6.952318,-0.961326,...,-10.934495,-0.276706,-1.342775,-8.850078,-0.552345,4.758396,6.654363,-6.527969,-18.863785,18.776985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05,-2.220283,6.538292,5.149391,5.458899,3.439343,4.936680,-0.544230,3.168266,2.240427,4.088098,...,4.665519,6.432951,6.569937,4.334324,3.934931,8.567450,8.968566,5.366876,11.439437,10.506614
2021-06,-8.693334,0.756567,0.509843,0.985625,3.989929,14.239408,0.684238,2.628481,0.029242,7.880865,...,-7.249836,-2.267316,-0.772450,0.801593,-2.030858,-0.249481,-0.569979,0.276944,6.066382,1.036007
2021-07,-5.485848,-5.424423,2.969777,0.099269,2.925880,8.223685,-6.194158,-2.813341,3.575157,12.892454,...,-5.086204,-9.408862,0.515354,2.868981,3.628709,-3.193194,-3.328503,3.176826,3.312402,-3.848731


In [72]:
# Average Value Weighted Returns
dict1_avwr = dict1[0]; dict1_avwr
r = np.log(1+dict1_avwr/100)*100;r

mom = r.shift(2); mom.head(12) # Shift 2는 데이터 들을 두 달 뒤로 옮기는 것/ 맨 뒤에꺼는 없어짐

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03,-4.604394,-10.903392,17.981843,-2.306394,-9.014355,-14.861602,3.691035,-0.621930,-6.731571,-12.216763,...,-14.098769,-10.048354,7.315752,-15.677078,-7.677306,-2.583076,-7.547814,-0.692392,-7.042246,0.039992
2000-04,7.844143,-7.343128,-8.642973,-12.375081,-4.092617,0.419120,-2.337099,-0.611868,-12.295891,-13.616342,...,-13.467490,-5.498431,1.744691,-4.009309,-12.817434,-11.991030,-13.650724,2.244619,6.344417,-2.173449
2000-05,4.171759,10.219551,-0.290421,0.129916,4.983723,7.408654,10.165365,12.327902,-15.385072,22.617894,...,11.716062,12.106653,6.793898,13.435605,14.824758,14.010971,20.595670,3.227356,13.723702,-2.020271
2000-06,-7.915144,-4.165568,-0.803217,3.613905,3.719944,0.269636,2.322813,-7.839407,4.200529,4.210118,...,-8.566683,3.420817,-2.634398,-5.762911,3.748844,-2.891401,-0.400802,-3.780571,-11.810809,8.837711
2000-07,-2.501016,16.602278,-7.515467,11.216730,17.822986,1.143438,1.252128,-6.549898,2.751789,-5.171444,...,-2.367813,-3.697524,1.291623,-3.087166,-4.301189,7.649797,7.055166,-2.747398,-7.052977,9.939274
2000-08,-0.944446,2.459505,1.054421,5.874050,2.917038,-1.694272,0.289580,-0.030005,-2.562555,-7.010063,...,-3.231660,-1.969263,-2.091725,-2.163230,-3.832511,-10.402807,-2.490764,-1.065658,13.549197,2.273949
2000-09,-3.138747,-1.602776,12.398598,6.363186,-4.572985,-10.569390,-0.702461,-1.126319,-2.675474,5.808021,...,-0.020002,6.391333,3.941300,-2.112150,-1.979463,8.065790,9.830569,3.004412,7.973497,-1.653597


In [70]:
mom = r.shift(2).rolling(2).sum() ; mom.head(20)

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04,3.239749,-18.246520,9.338870,-14.681475,-13.106972,-14.442482,1.353937,-1.233798,-19.027461,-25.833106,...,-27.566260,-15.546785,9.060443,-19.686387,-20.494740,-14.574106,-21.198539,1.552227,-0.697829,-2.133457
2000-05,12.015902,2.876423,-8.933394,-12.245165,0.891107,7.827775,7.828267,11.716034,-27.680963,9.001552,...,-1.751428,6.608222,8.538590,9.426296,2.007324,2.019941,6.944946,5.471975,20.068119,-4.193720
2000-06,-3.743384,6.053984,-1.093638,3.743820,8.703668,7.678290,12.488178,4.488495,-11.184543,26.828012,...,3.149379,15.527470,4.159501,7.672694,18.573602,11.119570,20.194868,-0.553215,1.912893,6.817440
2000-07,-10.416160,12.436711,-8.318685,14.830634,21.542930,1.413074,3.574941,-14.389305,6.952318,-0.961326,...,-10.934495,-0.276706,-1.342775,-8.850078,-0.552345,4.758396,6.654363,-6.527969,-18.863785,18.776985
2000-08,-3.445462,19.061783,-6.461046,17.090779,20.740024,-0.550834,1.541708,-6.579903,0.189234,-12.181506,...,-5.599473,-5.666787,-0.800102,-5.250396,-8.133700,-2.753010,4.564402,-3.813056,6.496220,12.213223
2000-09,-4.083193,0.856729,13.453019,12.237236,-1.655947,-12.263663,-0.412881,-1.156324,-5.238028,-1.202042,...,-3.251662,4.422069,1.849575,-4.275380,-5.811974,-2.337017,7.339805,1.938754,21.522694,0.620352


In [69]:
mom = r.shift(2).rolling(11).sum() ; mom.head(20)

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
mom = r.shift(2).rolling(11).sum()[12:] ; mom.head(20)

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2001-01,-17.637308,18.036377,28.297054,19.081487,54.787961,-15.918909,-10.789728,-9.578373,-21.350889,4.053025,...,-56.105607,4.006354,20.134188,-24.709199,-1.936516,10.589405,27.921869,0.707385,-1.409818,12.593869
2001-02,-1.575124,34.879804,5.385682,18.784280,79.519783,1.265506,-9.945193,0.683606,-11.517913,30.982395,...,-24.981940,16.260207,24.978233,-4.812416,8.102676,22.676222,41.964152,5.023326,21.392614,11.295999
2001-03,-3.450966,37.198810,16.243936,26.292843,83.712350,10.322843,8.075185,4.076444,-3.272976,48.174055,...,-4.664508,27.264258,21.335645,8.682446,17.087599,36.598483,44.611066,7.189964,23.509882,9.543377
2001-04,-13.544662,28.063361,23.505618,19.709075,87.785796,7.821773,-7.072250,-8.481722,11.026222,13.113153,...,-22.048188,8.254747,9.748694,-11.164354,2.222833,17.920272,23.302864,5.756423,-6.819279,12.805899
2001-05,-2.994540,29.430125,10.094859,6.234610,83.955791,6.728756,-19.299778,-7.438085,-2.451647,7.695777,...,-17.013140,3.687382,6.620180,-7.452330,-6.665868,19.056357,22.658220,8.005324,-8.865735,-4.380843
2001-06,-2.289556,10.797371,19.894049,-7.206680,71.572153,5.455234,-5.615123,4.047956,-3.448918,19.502159,...,-7.870117,10.447525,13.181945,0.494803,4.251541,14.129158,18.306188,14.530458,10.267312,-1.709729
2001-07,6.554490,10.943620,18.639428,-8.478286,70.899735,16.471214,3.717183,7.014422,4.353905,30.106834,...,-0.351667,16.914125,14.520843,4.284730,12.188650,28.895369,23.129535,20.093452,0.264481,-2.830354
2001-08,13.133380,11.843935,0.784651,-19.907663,74.901089,29.626870,2.184858,8.828371,7.875786,21.818301,...,-1.124802,7.363405,8.467394,3.865099,10.314818,21.646235,15.377220,20.422846,-9.984714,-1.899362
2001-09,14.972373,18.195591,2.054929,-7.637079,47.764891,24.348468,-11.718673,5.868492,8.684105,19.908042,...,3.529885,10.818526,-1.079935,14.537220,14.823455,12.760982,9.052448,20.292853,-32.278125,-24.787290


**1.5.** Now that we have a characteristic, we can form the sorted portfolios. Create $P=7$ portfolios sorted on your momentum. Compute the value weighted portfolio returns and store them in a new DataFrame `R_portf`.

*Note*: If you have portfolios sorted on momentum or reversal, these are resorted and grouped each month. If you have portfolios based on accouting numbers, these are resorted annually.

**1.6.** Compute the average return over time for each portfolio. Is there a pattern?

**1.7.** Now that we have our portfolios, we can set up the factor model. The monthly factors of the Fama-French three factor model (FF3) can be obtained from [Kenneth French's website](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html). 

Download the data set `'F-F_Research_Data_Factors'` from source `'famafrench'` via `pandas-datareader`. The output of DataReader is again a `dict` object.

**1.8.** Obtain the portfolios' excess returns by deducting the risk-free rate ('RF' in the DataFrame of the FF3 factors) from the portfolios returns.

**1.9.** Estimate the $\alpha_p$ and $\beta_p$ by using the following $P$ time series regressions:

$$
R_{pt}-R_{f,t} = \alpha_p + \beta_p' f_t + \varepsilon_{pt},
$$

where $\varepsilon_{pt} \sim N(0,\sigma^2_p)$, for $t = 1,\ldots,T$, $p = 1,\ldots,P$, where $f_t$ are your risk factors. In the case of the FF3 model, we have $f_t = (R_{mt}-R_{f,t},SMB_{t},HML_{t})'$.

Store the alpha estimates and $t$ statistics in a new DataFrame.

**1.10.** Inspect the alphas and $t$ statistics. Are there significant abnormal returns?

**1.11.** Make a scatter plot of the average return against the expected return (as implied by the asset pricing model).